<a href="https://colab.research.google.com/github/HausReport/ClubRaiders/blob/master/notebooks/History.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import sys

!{sys.executable} -m pip install --force-reinstall --upgrade "git+https://github.com/HausReport/ClubRaiders.git#egg=craid"
import craid.eddb.loader.DataProducer as dp
import pandas as pd
import logging
import plotly.graph_objs as go
import plotly.express as px
import numpy as np
from dateutil.relativedelta import relativedelta
from datetime import timedelta, date
import datetime as dt

logging.getLogger().addHandler(logging.StreamHandler())
logging.getLogger().level = logging.DEBUG

arrays = dp.getDataArrays()
baseFrame: pd.DataFrame = arrays['dataFrame']

  Cloning https://github.com/HausReport/ClubRaiders.git to /tmp/pip-install-7k5i1_y3/craid
  Running command git clone -q https://github.com/HausReport/ClubRaiders.git /tmp/pip-install-7k5i1_y3/craid
     |████████████████████████████████| 81kB 2.3MB/s 
     |████████████████████████████████| 3.4MB 8.4MB/s 
     |████████████████████████████████| 194kB 37.1MB/s 
     |████████████████████████████████| 1.8MB 42.8MB/s 
     |████████████████████████████████| 1.0MB 42.9MB/s 
     |████████████████████████████████| 102kB 10.9MB/s 
     |████████████████████████████████| 9.5MB 30.7MB/s 
     |████████████████████████████████| 71kB 7.4MB/s 
     |████████████████████████████████| 788kB 47.6MB/s 
     |████████████████████████████████| 14.5MB 141kB/s 
     |████████████████████████████████| 235kB 42.9MB/s 
     |████████████████████████████████| 184kB 44.2MB/s 
     |████████████████████████████████| 133kB 39.6MB/s 
     |████████████████████████████████| 471kB 42.4MB/s 
     |███████████████

Acquired lock.
Loading from AWS
Memory tracker: 0:169,930,752
1- downloading to: /tmp/smol-factions.jsonl.gz
2 - downloading [https://erlaed.s3.us-east-2.amazonaws.com/smol-factions.jsonl.gz] to [/tmp/smol-factions.jsonl.gz] data file.
Starting new HTTPS connection (1): erlaed.s3.us-east-2.amazonaws.com:443
https://erlaed.s3.us-east-2.amazonaws.com:443 "GET /smol-factions.jsonl.gz HTTP/1.1" 200 14843
Found data file: /tmp/smol-factions.jsonl.gz
Read 627 lines of faction data
Memory tracker: 0.5:171,130,880
1- downloading to: /tmp/smol-systems_populated.jsonl.gz
2 - downloading [https://erlaed.s3.us-east-2.amazonaws.com/smol-systems_populated.jsonl.gz] to [/tmp/smol-systems_populated.jsonl.gz] data file.
Starting new HTTPS connection (1): erlaed.s3.us-east-2.amazonaws.com:443
https://erlaed.s3.us-east-2.amazonaws.com:443 "GET /smol-systems_populated.jsonl.gz HTTP/1.1" 200 24285
Found data file: /tmp/smol-systems_populated.jsonl.gz
Read 166 lines of systems data
Memory tracker: 1:173,633

In [2]:
#pip install "pandas-profiling"


In [3]:
from craid.eddb.loader.strategy.DataLoader import DataLoader
from craid.eddb.loader.strategy.GithubLoader import LoadDataFromGithub
from craid.history.History import History

myLoader = LoadDataFromGithub(_forceWebDownload=True, useSmol=False)
hist = History()
csa = hist.getRawDataFrame()
target = hist.getNormalizedDataFrame()

1- downloading to: /tmp/history.jsonl.gz
2 - downloading [https://erlaed.s3.us-east-2.amazonaws.com/history.jsonl.gz] to [/tmp/history.jsonl.gz] data file.
Starting new HTTPS connection (1): erlaed.s3.us-east-2.amazonaws.com:443
https://erlaed.s3.us-east-2.amazonaws.com:443 "GET /history.jsonl.gz HTTP/1.1" 200 225034


Creating the object


Found data file: /tmp/history.jsonl.gz
NumExpr defaulting to 2 threads.
Read system        18824
faction       18824
updated       18824
influence     18824
control       18824
region        18824
population    18824
dtype: int64 lines of history data


In [4]:
def getScroller(fig):
  fig.update_layout(
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=1,
                     label="1m",
                     step="month",
                     stepmode="backward"),
                dict(count=6,
                     label="6m",
                     step="month",
                     stepmode="backward"),
                dict(count=1,
                     label="YTD",
                     step="year",
                     stepmode="todate"),
                dict(count=1,
                     label="1y",
                     step="year",
                     stepmode="backward"),
                dict(step="all")
            ])
        ),
        rangeslider=dict(
            visible=True
        ),
        type="date"
  ))

In [5]:
def getFactionInfluenceGraph(fac: str):
  hist = History()
  ogun = hist.getHistoryForFacation(fac)
  #ogun = target[ target['faction']==fac]
  a = ogun['system'].unique().tolist()
  fig = go.Figure()
  for sys in a:
    theSlice = ogun[ ogun['system'] == sys].reset_index()
    fig.add_trace(go.Scatter(x=theSlice['updated'], y=theSlice['influence'],
                    mode='lines',
                    stackgroup="b",
                    name=sys,
                    stackgaps='interpolate'))
  getScroller(fig)
  return fig

In [6]:
def getSystemInfluenceGraph(sys: str):
  #ogun = target[ target['system']==sys]
  hist = History()
  ogun = hist.getHistoryForSystem(sys)
  a = ogun['faction'].unique().tolist()
  fig = go.Figure()
  for fac in a:
    theSlice = ogun[ ogun['faction'] == fac].reset_index()
    fig.add_trace(go.Scatter(x=theSlice['updated'], y=theSlice['influence'],
                    mode='lines',
                    name=fac))
  getScroller(fig)
  return fig

In [7]:
def getClubInfluencePieChart():
  hist = History()
  stage1 = hist.getLatestInfluences() 
  stage3 = stage1[['system','faction','influence']].groupby('faction').influence.sum().reset_index()
  fig = px.pie(stage3, values='influence', names='faction', title='Total Influence by Minor Faction', width=350, height=350)
  fig.update_traces(textposition="inside")
  fig.update_layout(showlegend=False)
  return fig


In [8]:
hist.getLatestInfluences() 

,system,faction,updated,influence,control,region,population
18616,Abroin,Abroin Universal PLC,2020-10-23,47.65700,True,6,2813462
18413,Baiame,Abroin Universal PLC,2020-10-21,9.31860,False,6,20482
18190,Carunda,Abroin Universal PLC,2020-10-19,9.77070,False,6,122147
18311,HIP 66267,Abroin Universal PLC,2020-10-20,10.19170,False,6,71295
18742,HIP 67109,Abroin Universal PLC,2020-10-24,12.18780,False,6,69118
...,...,...,...,...,...,...,...
18744,HIP 80242,Wreaken Construction,2020-10-24,6.31900,False,9,19863348
18538,HR 5975,Wreaken Construction,2020-10-22,4.88530,False,9,4667556
3427,Jurati,Wreaken Construction,2020-04-01,3.16535,False,9,0
12903,LTT 6714,Wreaken Construction,2020-08-26,1.90380,False,9,5208044


In [9]:
fig = getClubInfluencePieChart()
fig.show()

In [10]:
from craid.club.regions.RegionFactory import RegionFactory
hist = History()
stage1 = hist.getLatestInfluences() 
stage3 = stage1[['region','influence']].groupby('region').influence.sum().reset_index()
dicts = [ {'region': 7, 'influence': 0.0}]
stage3 = stage3.append(dicts, ignore_index=True, sort=False)
stage3 = stage3.sort_values(['region'])
regionNames = list(RegionFactory.regionDict.keys())
fig = px.pie(stage3, values='influence', names=regionNames, title='Total Club Influence by Region', width=350, height=350) #, names='faction'
fig.update_traces(textposition="inside")
fig.update_layout(showlegend=False)
fig.show()

In [11]:
region = 1
regionNames = list(RegionFactory.regionDict.keys())
regionName = regionNames[region]
title = f"Total Influence <br>by Minor Faction <br>in the {regionName} region"
merope = hist.getHistoryForRegion(region) # target[ target['region']== region]
#stage1 = merope.loc[merope.groupby(['faction','system']).updated.idxmax()]
stage3 = merope[['system','faction','influence']].groupby('faction').influence.sum().reset_index()
fig = px.pie(stage3, values='influence', names='faction', title=title, width=350, height=350) #, names='faction'
fig.update_traces(textposition="inside")
fig.update_layout(showlegend=False)
fig.show()

In [12]:
theDate = '2020-06-1'
hist.getClubInfluenceOnDate(theDate)

4201.459213326396

In [13]:
theDate = '2020-10-22'
hist.getClubInfluenceOnDate(theDate)

4766.221713326396

In [14]:
region = 8

dates = []
inf = []

start_date = date(2018, 5, 15)
daily_date = date(2020,5,30)
end_date = date.today() + timedelta(days=1) 

dframe = hist.getHistoryForRegion(region)#   csa[ csa['region'] == region]
regionNames = list(RegionFactory.regionDict.keys())
regionName = regionNames[region]
title = f"Total Club Influence in the {regionName} region"

single_date = start_date
while single_date <= end_date:
    theDate = single_date.strftime("%Y-%m-%d")
    dated = dframe [ dframe['updated']<= theDate]

    stage1 = dated.loc[dated.groupby(['faction','system']).updated.idxmax()]
    stage3 = stage1[['system','faction','influence']].groupby('faction').influence.sum().reset_index()
    theInf = stage3.influence.sum()
    dates.append(theDate)
    inf.append(theInf)
    if(single_date<daily_date):
      single_date = single_date + relativedelta(months=1)
    else:
      single_date = single_date + relativedelta(days=1)
      
labels = {'x':'Day', 'y':"Total Influence"}
fig = px.line(x=dates, y=inf, title=title, labels=labels)
fig.show()

In [15]:

dates = []
inf = []

start_date = date(2018, 5, 15)
daily_date = date(2020,5,30)
end_date = date.today() + timedelta(days=1) 

dframe = hist.getHistoryForRegion(region)
regionNames = list(RegionFactory.regionDict.keys())
regionName = regionNames[region]
title = f"Total Club Influence"

single_date = start_date
while single_date <= end_date:
    theDate = single_date.strftime("%Y-%m-%d")
    dated = dframe [ dframe['updated']<= theDate]

    stage1 = dated.loc[dated.groupby(['faction','system']).updated.idxmax()]
    stage3 = stage1[['system','faction','influence']].groupby('faction').influence.sum().reset_index()
    theInf = stage3.influence.sum()
    dates.append(theDate)
    inf.append(theInf)
    if(single_date<daily_date):
      single_date = single_date + relativedelta(months=1)
    else:
      single_date = single_date + relativedelta(days=1)

labels = {'x':'Day', 'y':"Total Influence"}
fig = px.line(x=dates, y=inf, title=title, labels=labels)
fig.show()

In [16]:
dates = []
inf = []

faction = "Sirius Corporation"
faction = "Hodack Prison Colony"
faction = "Wreaken Construction"
#faction = "'Sirius Drives'"
faction = "Aegis Core"
#faction = "Aegis Research"
#faction = "Bill Turner"

start_date = date(2018, 5, 15)
daily_date = date(2020,5,30)
end_date = date.today() + timedelta(days=1) #date(2020, 6, 28)

dframe = hist.getHistoryForFacation(faction)
#dframe = getHistoryForFaction(faction)
regionNames = list(RegionFactory.regionDict.keys())
regionName = regionNames[region]
title = f"Total Influence of the {faction} Faction"

single_date = start_date
while single_date <= end_date:
    theDate = single_date.strftime("%Y-%m-%d")
    dated = dframe [ dframe['updated']<= theDate]

    stage1 = dated.loc[dated.groupby(['faction','system']).updated.idxmax()]
    stage3 = stage1[['system','faction','influence']].groupby('faction').influence.sum().reset_index()
    theInf = stage3.influence.sum()
    dates.append(theDate)
    inf.append(theInf)
    if(single_date<daily_date):
      single_date = single_date + relativedelta(months=1)
    else:
      single_date = single_date + relativedelta(days=1)


labels = {'x':'Day', 'y':"Total Influence"}
fig = px.line(x=dates, y=inf, title=title, labels=labels)
fig.show()

In [17]:
getFactionInfluenceGraph("Aegis Core").show()

In [18]:
badGuys= ['Abroin Universal PLC',
                     'Aegis Core', 'Aegis Defense', 'Aegis Research',
                      'Bill Turner',
                     'CQC Holdings',
                     'Gallant Investment Brokers', 'Hodack Prison Colony',
                     'Janus Incorporated', "Namarii Emperor's Dawn",
                     'Pleiades Resource Enterprise',
                     'Reyan BPS', 'Reynhardt IntelliSys',
                     'Sirius Atmospherics', 'Sirius Catering', 'Sirius Corporation',
                     'Sirius Drives', 'Sirius Hot2Cold', 'Sirius Hyperspace',
                     'Sirius Industrial', 'Sirius Luxury Transports', 'Sirius Mining Merope',
                     'Sirius Mining', 'Sirius Power',
                     'The Greenventure Group',
                     'The Peterson Group', 'The Rockforth Corporation',
                     'Turner Research Group', 'Wiggins Development Trust',
                     'Wreaken Construction'
                     ]

In [19]:
a = csa['faction'].unique().tolist()
b = baseFrame['factionName'].unique().tolist()
aa = set(a)
bb = set(b)
bb-aa

set()

In [20]:
dates = []
inf = []

faction = "Combined Sirius Factions"
faction1 = "Sirius"

start_date = date(2018, 5, 15)
daily_date = date(2020,5,30)
end_date = date.today() + timedelta(days=1) #date(2020, 6, 28)


dframe = target[ (target['faction'].str.contains(faction1)) ]
regionNames = list(RegionFactory.regionDict.keys())
regionName = regionNames[region]
title = f"Total Influence of the {faction} Faction"

single_date = start_date
while single_date <= end_date:
    theDate = single_date.strftime("%Y-%m-%d")
    dated = dframe [ dframe['updated']<= theDate]

    stage1 = dated.loc[dated.groupby(['faction','system']).updated.idxmax()]
    stage3 = stage1[['system','faction','influence']].groupby('faction').influence.sum().reset_index()
    theInf = stage3.influence.sum()
    dates.append(theDate)
    inf.append(theInf)
    if(single_date<daily_date):
      single_date = single_date + relativedelta(months=1)
    else:
      single_date = single_date + relativedelta(days=1)


labels = {'x':'Day', 'y':"Total Influence"}
fig = px.line(x=dates, y=inf, title=title, labels=labels)
fig.show()

In [21]:
# One System, handles Multiple Factions
fig = getSystemInfluenceGraph("Celaeno")
fig.show()

In [22]:
fig = getSystemInfluenceGraph("Solati")
fig.show()

In [23]:
# One System, handles Multiple Factions
fig = getSystemInfluenceGraph("Sol")
fig.show()

In [24]:
getFactionInfluenceGraph('Hodack Prison Colony').show()

In [25]:
fac = 'Sirius Mining'

fac = 'CQC Holdings'
fac = 'Aegis Core'
fac = 'Sirius Mining Merope'

getFactionInfluenceGraph(fac).show()

In [26]:
from typing import List
def getCombinedFactionGraph( factions: List[str], titleSubject=None):
  hist = History()
  labels = {'x':'Day', 'y':"Total Influence"}
  fig = go.Figure()
  for fac in factions:
    foo = hist.getHistoryForFacation(fac)    #target[target['faction']== fac]
    stage3 = foo[['system','faction','influence','updated']].groupby(['updated']).influence.sum().reset_index()
    fig.add_trace(go.Scatter(x=stage3['updated'], y=stage3['influence'],
                    mode='lines',
                    stackgroup="b",
                    name=fac,
                    stackgaps="interpolate"))

  if titleSubject is None:
    titleSubject = "+".join(factions)

  title = f"Total influence of {titleSubject}"
  fig.update_layout(title=title,
                    xaxis_title="Date",
                    yaxis_title="Total Influence")
  getScroller(fig)
  return fig



In [27]:
factions = ["Wreaken Construction", "Hodack Prison Colony"]

fig = getCombinedFactionGraph( factions, titleSubject=None)
fig.show()

In [28]:
siriusFactions = ['Sirius Atmospherics', 'Sirius Catering',
       'Sirius Corporation', 'Sirius Drives', 'Sirius Hot2Cold',
       'Sirius Hyperspace', 'Sirius Industrial',
       'Sirius Luxury Transports', 'Sirius Mining Merope',
       'Sirius Mining', 'Sirius Power']
      
fig = getCombinedFactionGraph( siriusFactions, titleSubject="Combined Sirius Factions")
fig.show()

In [29]:
pd.set_option('mode.chained_assignment', None)

#from craid.eddb.loader.strategy.AWSLoader import LoadDataFromAWS
#loader = LoadDataFromAWS(forceWebDownload=False, useSmol=False)
#fName = loader.find_data_file('history.jsonl')
#dataframe = pd.read_json(fName, lines=True, compression='infer')
#dataframe = dataframe[ dataframe.faction != "Benton's Gang"]
#dataframe = dataframe[ dataframe.faction != "Bentonian Party"]
#dataframe = dataframe[ dataframe.faction != "Worster Insurance"]

pd.set_option('max_rows', 99999)
pd.set_option('max_colwidth', 400)

#dataframe = dataframe.rename(columns={'string':'system',
#                          'string':'faction',
#                          'datetime': 'updated', 
#                          'float':'influence',
#                          'boolean':'control',
#                          'int':'region',
#                          'float': 'population'})

In [30]:
getFactionInfluenceGraph("Sirius Mining Merope").show()

In [31]:
getFactionInfluenceGraph("Sirius Hyperspace").show()

In [32]:
getFactionInfluenceGraph("Pleiades Resource Enterprise").show()

In [33]:
def factionPresenceGraph( faction: str):

  #
  # Narrow down part of dataframe you want here
  #
  #mf = dataframe.copy()
  mf = hist.getHistoryForFacation(faction).copy() #  dataframe[dataframe['faction']== faction].copy()


  #
  # Convert to monthly and clean
  #
  mf['updated'] = pd.to_datetime(mf.updated, unit='ms')
  mf['updated'] = pd.to_datetime(mf.updated).dt.to_period('W').dt.to_timestamp()
  mf = mf.drop_duplicates(subset=['system', 'faction', 'updated'])
  hurm = mf.groupby(['updated','faction']).count().reset_index()

  labels = {'x':'Day', 'y':"Total Systems"}
  #foo = hurm[hurm['faction']== faction]
  foo = hurm
  fig = go.Figure(data=go.Scatter(x=foo.updated, y=foo.system, mode='lines+markers'))

  #fig = go.Figure()
  #factions = hurm['faction'].unique().tolist()
  #for fac in factions:
  #  foo = hurm[hurm['faction']== fac]
  #  stage3 = foo[['system','faction','updated']].groupby(['updated']).count().reset_index()
  #  fig.add_trace(go.Scatter(x=stage3['updated'], y=stage3['count'],
  #                  mode='lines',
  #                  stackgroup="b",
  #                  name=fac,
  #                  stackgaps="interpolate"))

  #if titleSubject is None:
  #  titleSubject = "+".join(factions)

  title = f"System Presences of {faction}"
  fig.update_layout(title=title,
                    xaxis_title="Date",
                    yaxis_title="Total Systems")
  #getScroller(fig)
  return fig

In [34]:
factionPresenceGraph("Pleiades Resource Enterprise")

In [35]:
factionPresenceGraph("Sirius Mining Merope")

In [36]:
factionPresenceGraph("Sirius Hyperspace")

In [37]:
def regionPresenceGraph( faction: str, region: int):
  hist = History()
  #
  # Narrow down part of dataframe you want here
  #
  mf = hist.getHistoryForRegion(region).copy()
  #mf = dataframe[dataframe['region']== region].copy()


  #
  # Convert to monthly and clean
  #
  mf['updated'] = pd.to_datetime(mf.updated, unit='ms')
  mf['updated'] = pd.to_datetime(mf.updated).dt.to_period('W').dt.to_timestamp()
  mf = mf.drop_duplicates(subset=['system', 'faction', 'updated'])
  hurm = mf.groupby(['updated']).count().reset_index()

  labels = {'x':'Day', 'y':"Total Systems"}
  #foo = hurm[hurm['faction']== faction]
  foo = hurm
  fig = go.Figure(data=go.Scatter(x=foo.updated, y=foo.system, mode='lines+markers'))

  #fig = go.Figure()
  #factions = hurm['faction'].unique().tolist()
  #for fac in factions:
  #  foo = hurm[hurm['faction']== fac]
  #  stage3 = foo[['system','faction','updated']].groupby(['updated']).count().reset_index()
  #  fig.add_trace(go.Scatter(x=stage3['updated'], y=stage3['count'],
  #                  mode='lines',
  #                  stackgroup="b",
  #                  name=fac,
  #                  stackgaps="interpolate"))

  #if titleSubject is None:
  #  titleSubject = "+".join(factions)

  title = f"System Presences of {faction}"
  fig.update_layout(title=title,
                    xaxis_title="Date",
                    yaxis_title="Total Systems")
  #getScroller(fig)
  return fig

In [38]:
regionPresenceGraph("Sirius Hyperspace",2)

In [39]:
def allClubFactionsGraph( ):

  hist = History()
  mf = hist.getHistoryByWeek().copy()
  hurm = mf.groupby(['updated','faction']).count().reset_index()

  labels = {'x':'Day', 'y':"Total Systems"}


  fig = go.Figure()
  factions = hurm['faction'].unique().tolist()
  for fac in factions:
    foo = hurm[ (hurm['faction']== fac) & (hurm['updated'] > '2019/12/01') ]
    fig.add_trace(go.Scatter(x=foo['updated'], y=foo['system'],
                    mode='lines',
                    stackgroup="b",
                    name=fac,
                    stackgaps="interpolate"))

  #if titleSubject is None:
  #  titleSubject = "+".join(factions)

  title = f"Club System Presences"
  fig.update_layout(title=title,
                    xaxis_title="Date",
                    yaxis_title="Total Systems")
  getScroller(fig)
  return fig

In [40]:
allClubFactionsGraph( )

In [41]:

df2

,system,faction,updated,influence,control,region,population
16863,Mooramba,Sirius Hyperspace,2020-10-06,57.4426,True,2,2752
16959,Mooramba,Sirius Hyperspace,2020-10-07,47.6048,True,2,2752
17059,Mooramba,Sirius Hyperspace,2020-10-08,38.0808,True,2,2752
17156,Mooramba,Sirius Hyperspace,2020-10-09,33.8726,True,2,2752
17369,Mooramba,Sirius Hyperspace,2020-10-11,33.8726,True,2,2752
17591,Mooramba,Sirius Hyperspace,2020-10-13,33.8726,True,2,2752
17696,Mooramba,Sirius Hyperspace,2020-10-14,34.5691,True,2,2752
17901,Mooramba,Sirius Hyperspace,2020-10-16,34.6038,True,2,2752
18121,Mooramba,Sirius Hyperspace,2020-10-18,24.0240,False,2,2752
18229,Mooramba,Sirius Hyperspace,2020-10-19,18.1000,False,2,2752


In [68]:
from craid.eddb.faction.Strategy import Strategy
from craid.eddb.faction.Simulation import Simulation


def getNiceSimulation(startDateString, ally,target,allyInf, targetInf, sys, pop):
  date_time_obj = dt.datetime.strptime(startDateString, '%Y-%m-%d')
  df2 = hist.getSmallFrame(startDateString,target,sys)

  strat = Strategy()
  theSim = Simulation(strat)
  sim = theSim.getDatedSimulationFrame( date_time_obj, allyInf, targetInf,pop)
  fig = theSim.getSimulationFigure(sim,title=f"Retreat of {target} in {sys}",ally=ally,target=target)
  fig.add_trace(go.Scatter(x=df2.updated, y=df2.influence,
                                 mode='lines',
                                 name=f'{target} Actual',
                                 marker_color='magenta'))
  return fig

fig = getNiceSimulation("2020-10-05","VTC","Sirius Hyperspace",6.89,57.44,"Mooramba",2752)
fig.show()

In [69]:
fig = getNiceSimulation("2020-10-01","AT","Sirius Hyperspace",15.62,41.8,"Daik",2200)
fig.show()

In [48]:
#hist.getRawDataFrame()
#hist.getHistoryForFacationAndSystem("Sirius Hyperspace","Daik")
#hist.getLatestInfluences()
#hist.getHistoryByWeek()
#hist.getHistoryByMonth()


In [49]:
# These are PROBABLE/POSSIBLE RETREATS
stage1 = hist.getLatestInfluences()
stage1 = stage1[ stage1['updated']< "10/1/2020"].sort_values("updated")
stage1

,system,faction,updated,influence,control,region,population
11062,HIP 22460,Pleiades Resource Enterprise,2017-10-02,100.000000,False,1,0
970,LTT 6830,CQC Holdings,2018-05-01,0.979400,False,10,0
1389,LTT 4497,Hodack Prison Colony,2018-05-01,5.577886,False,8,0
1223,Codorain,Hodack Prison Colony,2018-06-01,2.100900,False,8,0
1832,Pleiades Sector HR-W d1-57,Janus Incorporated,2018-07-01,1.024738,False,1,0
2596,Malanquiya,Sirius Drives,2018-08-01,29.550000,False,0,0
279,HIP 16753,Aegis Research,2018-09-01,2.095500,False,1,0
1970,Kamcha,Namarii Emperor's Dawn,2018-09-01,3.356886,False,0,0
431,HR 1185,Aegis Research,2018-11-01,1.096700,False,1,0
440,Hyades Sector AB-W b2-2,Aegis Research,2019-01-01,3.236160,False,0,0


In [50]:
#csa = hist.getRawDataFrame()
#rec = { "system":"Cail", "faction":"Sirius Hyperspace",	"updated":"2020-05-02 00:00:00",	"influence":0.0,	"control":"False",	"region":2,	"population":0 }
#csa = csa.append(rec, ignore_index=True, sort=False)
#csa[ csa['system']=="Merope"]



In [75]:
system = 'Merope'
faction = 'Aegis Research'
def getCappingCode(faction, system):
  tmp = hist.getLatestInfluences()
  tmp = tmp[ ( tmp['system']==system) & ( tmp['faction']==faction) ]
  #return tmp

  cont = tmp['control'].values[0]
  reg = tmp['region'].values[0]
  pop = tmp['population'].values[0]
  updated = tmp['updated'].values[0]
  ret = {'system':f'{system}', 'faction':f'{faction}', 'influence':0.0, 'control':'False', 'region':f'{reg}', 'population':f'{pop}'}
  #print(ret)
  return ret

print(getCappingCode(faction,system))
#foo = getCappingCode(faction, system)
#csa = csa.append(foo, ignore_index=True, sort=False)

{'system': 'Merope', 'faction': 'Aegis Research', 'influence': 0.0, 'control': 'False', 'region': '1', 'population': '0'}
